In [1]:
# author: Zhifei Wang, 11.14, 2021

import socket
import cv2
import numpy as np
import time
import sys 

# HDMI out
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("base.bit")
hdmi_out = base.video.hdmi_out
Mode = VideoMode(640,480,24)
hdmi_out.configure(Mode,PIXEL_BGR)

In [2]:
def Connect(sock):
    try:
        sock.connect(address)
    except socket.error as msg:
        print(msg)

def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf

def SendVideo(frame,sock):

    result, imgencode = cv2.imencode('.jpg', frame, encode_param)
    
    data = np.array(imgencode)
    stringData = data.tostring()

    sock.send(str.encode(str(len(stringData)).ljust(16)))
    sock.send(stringData)

    time.sleep(0.05)
    
    length = recvall(sock,16)
    receive = recvall(sock, int(length))
    #receive = sock.recv(1024)
    data = np.frombuffer(receive, np.uint32)
    if len(data): 
        # print(str(data))
        return data
    


In [ ]:
if __name__ == '__main__':

    address = ('192.168.137.1', 8001)
    sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    
    # img encode param
    encode_param=[int(cv2.IMWRITE_JPEG_QUALITY),15] 

    conn=Connect(sock)
    
    capture = cv2.VideoCapture(0)
    
    hdmi_out.start()
    
    board=np.ones((480,640),np.uint8)
    board*=255
    
    last_point=[]
    currunt_point=[]
    
    ret, frame = capture.read()
    
    while ret:
        ret, frame = capture.read()
        
        data=SendVideo(frame,sock)
        
        if (data[0]==0): # write
            cv2.putText(frame, 'Writing', (10, 30),cv2.FONT_HERSHEY_COMPLEX,1,(0, 255, 0),1,4)
            currunt_point = (data[1],data[2])
            if (last_point):
                cv2.line(board, last_point, currunt_point, (0), 4, 4)
                last_point=currunt_point
            else:
                last_point=currunt_point
                
        elif(data[0]==1): # erase
            cv2.putText(frame, 'Erasing', (10, 30),cv2.FONT_HERSHEY_COMPLEX,1,(0, 255, 0),1,4)
            cv2.rectangle(frame, (data[1],data[2]), (data[3],data[4]), (255,0,0),4)
            
            board[data[2]:data[4],data[3]:data[1]]=np.logical_or(board[data[2]:data[4],data[3]:data[1]],1)*255
            #print('y1:',data[2],'y2:',data[4],'x1:',data[1],'x2:',data[3])
            last_point=[]
            
        else: # without hands
            cv2.putText(frame, 'No hands', (10, 30),cv2.FONT_HERSHEY_COMPLEX,1,(255, 0, 0),1,4)
            last_point=[]
            
        frame=cv2.bitwise_and(frame, frame,mask=board)
        
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame[0:480,640::-1,:]
        hdmi_out.writeframe(outframe)
        
        if cv2.waitKey(10) == 27: 
            break
            sock.close()